In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col, when

df = spark.read.table("imdb_dev.bronze.title_episode") 

df_to_fill = df.filter((col("seasonNumber").isNull()))
df_to_fill = df_to_fill.withColumn("episodeNumber", when(col("seasonNumber").isNull(), row_number().over(Window.partitionBy("parentTconst").orderBy("tconst"))).otherwise(col("seasonNumber"))) 

df_filled = df.filter(col("seasonNumber").isNotNull())

df_episode = df_filled.unionByName(df_to_fill)
df_episode = (
    df_episode.withColumn("seasonNumber", when(col("seasonNumber").isNull(), 1).otherwise(col("seasonNumber")))
    .withColumnRenamed("tconst", "titleId")
    .withColumnRenamed("parentTconst", "parentTitleId") 
)

df_episode.write.format("delta").mode("overwrite").insertInto("imdb_dev.silver.TitleEpisode")